# EDA on commute data

In [2]:
import pandas as pd
import numpy as np

In [56]:
# load data
table1 = pd.read_excel(
    '../data/commute/table1.xlsx',
    #header=5,
    skiprows=[0,1,2,3,4,5, 139440, 139441],
    dtype=str
)
table1.columns = [
    'Resident State FIPS',
    'Resident County FIPS',
    'Resident State Name',
    'Resident County Name',
    'Work State FIPS',
    'Work County FIPS',
    'Work State Name',
    'Work County Name',
    'commute',
    'error',
]

In [83]:
# load data: county info
county_info = pd.read_csv('../data/02_county_FIPS.csv', dtype=str, index_col=0)

In [57]:
# preprocess the table : change the double float type
table1['commute'] = table1['commute'].astype(np.float)
table1['error'] = table1['error'].astype(np.float)

In [58]:
# preprocess: change the county FIPS
table1['Work State FIPS'] = table1['Work State FIPS'].apply(lambda x : x[1:] if type(x) == str else np.nan)
table1['Resident County FIPS'] = table1['Resident State FIPS'] + table1['Resident County FIPS']
table1['Work County FIPS'] = table1['Work State FIPS'] + table1['Work County FIPS']

In [64]:
# visualize the california data
table1[table1['Resident State Name'] == 'California']

,Resident State FIPS,Resident County FIPS,Resident State Name,Resident County Name,Work State FIPS,Work County FIPS,Work State Name,Work County Name,commute,error
6585,06,06001,California,Alameda County,01,01089,Alabama,Madison County,4.0,6.0
6586,06,06001,California,Alameda County,04,04013,Arizona,Maricopa County,106.0,48.0
6587,06,06001,California,Alameda County,04,04019,Arizona,Pima County,31.0,37.0
6588,06,06001,California,Alameda County,05,05007,Arkansas,Benton County,13.0,21.0
6589,06,06001,California,Alameda County,06,06001,California,Alameda County,462270.0,3414.0
...,...,...,...,...,...,...,...,...,...,...
10412,06,06115,California,Yuba County,32,32031,Nevada,Washoe County,65.0,71.0
10413,06,06115,California,Yuba County,41,41027,Oregon,Hood River County,7.0,13.0
10414,06,06115,California,Yuba County,45,45083,South Carolina,Spartanburg County,5.0,8.0
10415,06,06115,California,Yuba County,48,48029,Texas,Bexar County,19.0,27.0


In [62]:
# visualize the states with the most people go to work
table1.groupby('Work State Name')['commute'].sum().sort_values(ascending=False).head()

Work State Name
California    16864403.0
Texas         11961894.0
New York       9417672.0
Florida        8385355.0
Illinois       5971813.0
Name: commute, dtype: float64

In [63]:
# visualize the states with the most people resides
table1.groupby('Resident State Name')['commute'].sum().sort_values(ascending=False).head()

Resident State Name
California    16869052.0
Texas         11966019.0
New York       9064986.0
Florida        8432513.0
Illinois       5983836.0
Name: commute, dtype: float64

In [85]:
missed_counties

{'02198', '02230', '02275', '15005', '46102', '72039', '72069'}

In [87]:
# preprocess find the county FIPS that are in this data but not in county_info
missed_counties = set(table1['Resident County FIPS'].unique()) - set(county_info['FIPS'])
table1.loc[table1['Resident County FIPS'].isin(missed_counties), 'Resident County Name'].unique()

array(['Prince of Wales-Hyder Census Area', 'Skagway Municipality',
       'Wrangell City and Borough', 'Kalawao County',
       'Oglala Lakota County', 'Ciales Municipio', 'Humacao Municipio'],
      dtype=object)